```json
            { 
                "tag": "q_0000_0001", 
                "type": "question", 
                "description": { 
                    "number": "", 
                    "question": "", 
                    "options": [], 
                    "options": null, 
                    "answer": "", 
                    "explanation": "" 
                    }, 
                "caption": [ 
                    "" 
                    ], 
                "file_path": null, 
                "bbox": null 
            }, 
{ 
    "tag": "note_0000_0001", 
    "type": "footnote", 
    "description": "1 이러한 계약을 법적으로는 금전소비대차계약이라고 한다. 차용증서를 영어로는 I owe you.의 소리를 따라 IOU 라고 한다.", 
    "caption": null, 
    "file_path": null, 
    "bbox": null 
}, 
{ 
    "tag": "tb_0000_0001", 
    "type": "table", 
    "description": "", 
    "caption": [], 
    "file_path": "??????/crop/tb_0000_0001.png", 
    "bbox": null 
}, 
{ 
    "tag": "img_0000_0001", 
    "type": "image", 
    "description": null, 
    "caption": [], 
    "file_path": "?????/crop/img_0000_0001.png", 
    "bbox": null 
}
```

### 도서 정보 추출

In [5]:
import tools.ProcessFiles as pf
import pandas as pd
excel = pd.concat([pf.get_excel_data(1), pf.get_excel_data(2), pf.get_excel_data(3)]) # i차 분석
excel

,ISBN,도서명,출판일,코퍼스 1분류,코퍼스 2분류,비고,분류
관리번호,,,,,,,
SS0319,9791163478782,금융의 현재와 미래,2020-01-03,금융,기초,시사,Lv3/4
SS0320,9788984059900,요즘 금리 쉬운 경제,2020-04-27,금융,기초,시사,Lv3/4
SS0321,9788960542570,알기 쉬운 보험 세테크 100% 활용법,2020-06-23,보험,기초,세무,Lv3/4
SS0322,9788957614358,공정거래 용어사전,2020-08-10,금융,기초,용어,Lv2
SS0323,9791190820660,1페이지 보험 정리의 기술,2020-08-28,보험,기초,시사,Lv3/4
...,...,...,...,...,...,...,...
SS0314,9791193595244,리스크 관리,2025-02-01,증권,실무,리스크관리,Lv3/4
SS0315,9791194530152,환율의 대전환,2025-03-01,금융,기초,시사,Lv3/4
SS0316,9791157848140,빅 사이클,2025-06-03,금융,기초,시사,Lv3/4


['SS0001',
 'SS0002',
 'SS0003',
 'SS0004',
 'SS0005',
 'SS0006',
 'SS0007',
 'SS0008',
 'SS0009',
 'SS0010',
 'SS0011',
 'SS0012',
 'SS0013',
 'SS0014',
 'SS0015',
 'SS0016',
 'SS0017',
 'SS0018',
 'SS0019',
 'SS0020',
 'SS0021',
 'SS0022',
 'SS0023',
 'SS0024',
 'SS0025',
 'SS0026',
 'SS0027',
 'SS0028',
 'SS0029',
 'SS0030',
 'SS0031',
 'SS0032',
 'SS0033',
 'SS0034',
 'SS0035',
 'SS0036',
 'SS0037',
 'SS0038',
 'SS0039',
 'SS0040',
 'SS0041',
 'SS0042',
 'SS0043',
 'SS0044',
 'SS0045',
 'SS0046',
 'SS0047',
 'SS0048',
 'SS0049',
 'SS0050',
 'SS0051',
 'SS0052',
 'SS0053',
 'SS0054',
 'SS0055',
 'SS0056',
 'SS0057',
 'SS0058',
 'SS0059',
 'SS0060',
 'SS0061',
 'SS0062',
 'SS0063',
 'SS0064',
 'SS0066',
 'SS0069',
 'SS0071',
 'SS0072',
 'SS0075',
 'SS0077',
 'SS0078',
 'SS0080',
 'SS0081',
 'SS0083',
 'SS0085',
 'SS0087',
 'SS0090',
 'SS0092',
 'SS0093',
 'SS0095',
 'SS0098',
 'SS0099',
 'SS0100',
 'SS0101',
 'SS0103',
 'SS0104',
 'SS0105',
 'SS0106',
 'SS0107',
 'SS0108',
 'SS0109',

### Lv2 파일명 변경, Lv3/4/5 폴더 분류

In [ ]:
import pandas as pd
import os

base_path = "/Users/jinym/Library/CloudStorage/OneDrive-개인/데이터L/selectstar"
analysis = {1:'1차 분석', 2:'2차 분석', 3: '3차 분석'}
buy = {1:'1차 구매', 2:'2차 구매', 3: '3차 구매'}
i = 1

excel_analy = pd.read_excel(os.path.join(base_path, 'book_list_ALL.xlsx'), sheet_name=analysis[i], header=3)[['관리번호_old', '관리번호', 'ISBN', '도서명', '분류']]

In [ ]:
import shutil
from tqdm import tqdm

zip_path = "/Users/jinym/Library/CloudStorage/OneDrive-개인/데이터L/selectstar/data/zip/250826_1차_원천코퍼스_보완사항반영"
original_path = f"/Users/jinym/Library/CloudStorage/OneDrive-개인/데이터L/selectstar/data/ORIGINAL/{i}C"

error_list = []

for fn, tn, isbn, lv in tqdm(zip(excel_analy['관리번호_old'], excel_analy['관리번호'], excel_analy['ISBN'], excel_analy['분류'])):
   fn = str(fn)
   if '/' in lv: 
      lv = lv.replace("/", '_')
   
   if 'Lv' not in lv:
      continue

   # os.system(f"cd /Users/yejin/Library/CloudStorage/OneDrive-개인/데이터L/selectstar/data/ORIGINAL/1C/Lv2 && mv {isbn}.json {tn}.json")
   if lv == 'Lv2':
      # os.system(f"cd /Users/yejin/Library/CloudStorage/OneDrive-개인/데이터L/selectstar/data/ORIGINAL/1C/Lv2 && mkdir {tn} && mv {tn}.json {tn}/")
      continue
   else:
      os.system(f"cp {zip_path}/{fn}.zip {original_path}/{lv}/{tn}.zip && cd {original_path}/{lv} && mkdir {tn} && unzip {tn}.zip -d {tn} && find . -type f -name '*{fn}*'| sed -e 'p' -e 's/{fn}/{tn}/g' | xargs -n 2 mv")

      file_path = f"{original_path}/{lv}/{tn}/{tn}.json"
      try:
         with open(file_path, 'r', encoding='utf-8') as f:
               content = f.read()
         
         # Replace the old filename with new filename
         updated_content = content.replace(str(fn), str(tn))
         
         with open(file_path, 'w', encoding='utf-8') as f:
               f.write(updated_content)
               
         print(f"Updated {fn} to {tn} in {file_path}")
      except Exception as e:
         print(f"Error processing {fn} to {tn}: {e}")
         error_list.append((fn, tn, lv))
         # 이름 바꾸기
         # os.system(f"cd {original_path}/{lv} && mkdir {tn} && unzip {tn}.zip -d {tn} && find . -type f -name '*{fn}*'| sed -e 'p' -e 's/{fn}/{tn}/g' | xargs -n 2 mv")

In [ ]:
import shutil
from tqdm import tqdm

zip_path = "/Users/jinym/Library/CloudStorage/OneDrive-개인/데이터L/selectstar/data/zip/250826_1차_원천코퍼스_보완사항반영"
original_path = f"/Users/jinym/Library/CloudStorage/OneDrive-개인/데이터L/selectstar/data/ORIGINAL/{i}C"

error_list = []

for fn, tn, isbn, lv in tqdm(zip(excel_analy['관리번호_old'], excel_analy['관리번호'], excel_analy['ISBN'], excel_analy['분류'])):
   fn = str(fn)
   if '/' in lv: 
      lv = lv.replace("/", '_')
   
   if 'Lv' not in lv:
      continue

   # os.system(f"cd /Users/yejin/Library/CloudStorage/OneDrive-개인/데이터L/selectstar/data/ORIGINAL/1C/Lv2 && mv {isbn}.json {tn}.json")
   if lv == 'Lv2':
      # os.system(f"cd /Users/yejin/Library/CloudStorage/OneDrive-개인/데이터L/selectstar/data/ORIGINAL/1C/Lv2 && mkdir {tn} && mv {tn}.json {tn}/")
      continue
   else:
    #   os.system(f"cp {zip_path}/{fn}.zip {original_path}/{lv}/{tn}.zip && cd {original_path}/{lv} && mkdir {tn} && unzip {tn}.zip -d {tn} && find . -type f -name '*{fn}*'| sed -e 'p' -e 's/{fn}/{tn}/g' | xargs -n 2 mv")
      os.system(f"cd {original_path}/{lv}/{tn} && find . -type f -name '*{fn}*'| sed -e 'p' -e 's/{fn}/{tn}/g' | xargs -n 2 mv")

      file_path = f"{original_path}/{lv}/{tn}/{tn}.json"
      try:
         with open(file_path, 'r', encoding='utf-8') as f:
               content = f.read()
         
         # Replace the old filename with new filename
         updated_content = content.replace(str(fn), str(tn))
         
         with open(file_path, 'w', encoding='utf-8') as f:
               f.write(updated_content)
               
         print(f"Updated {fn} to {tn} in {file_path}")
      except Exception as e:
         print(f"Error processing {fn} to {tn}: {e}")
         error_list.append((fn, tn, lv))
         # 이름 바꾸기
         # os.system(f"cd {original_path}/{lv} && mkdir {tn} && unzip {tn}.zip -d {tn} && find . -type f -name '*{fn}*'| sed -e 'p' -e 's/{fn}/{tn}/g' | xargs -n 2 mv")

In [ ]:
import os

# from_path = "/Users/jinym/Library/CloudStorage/OneDrive-개인/데이터L/selectstar/evaluation/eval_data/mock_exam/test" 
from_path = '/Users/jinym/Desktop/Desktop_AICenter✨/SFAIcenter/tools/QueryModels.py'
to_path = "yjmoon@oci004:/home/yjmoon/src"


# 업로드
os.system(f"scp -r {from_path} {to_path}")
# for ss in file:
    # print(ss)
    # os.system(f"scp -r /Users/jinym/Library/CloudStorage/OneDrive-개인/데이터L/selectstar/data/FINAL/3C/Lv3_4/{ss} yjmoon@oci004:/home/yjmoon/FINAL/{ss}")

In [ ]:
for fn, tn, lv in zip(excel_analy['관리번호'], excel_analy['관리번호_new'], excel_analy['분류']):
    fn = str(fn)
    if '/' in lv: 
       lv = lv.replace("/", '_')
#     print(fn, tn, lv)
    
    dir = os.path.join(pf.FINAL_DATA_PATH, pf.CYCLE_PATH[1], lv)
    os.system(f'cd {dir} && mv {fn}_low {tn}_low && cd {tn} &&'+f' for file in *{fn}*; do mv "$file" "$'+'{file//'+fn+'/'+tn+'}"; done')
    print(os.path.join(dir, fn))
    # break

In [ ]:
import tools.ProcessFiles as pf

pf.move_jsons(3, pf.ORIGINAL_DATA_PATH)

---

### 개별 처리
- 불필요 페이지 제거
- 오타
- 페이지 머리말/꼬리말

## 전체 돌리기

In [ ]:
import os, json
from typing import List, Dict, Any
import tools.ProcessFiles as pf
import tools.ProcessLv2 as pl2

json_files = pf.get_filelist(1)
json_files

In [ ]:
# import os, json, shutil
# import tools.ProcessFiles as pf

# i = 3
# # 폴더 만들어 옮기기
# excel = pf.get_excel_data(i)
# FINAL_DATA_PATH = os.path.join("/Users/jinym/Desktop/Desktop_AICenter✨/SFAIcenter", '3C')
# ORIGINAL_DATA_PATH = os.path.join("/Users/jinym/Library/CloudStorage/OneDrive-개인/데이터L/selectstar/data/FINAL/", '3C')

# json_files = pf.get_filelist(i)

# for j in json_files:
#     new_path = j.replace(ORIGINAL_DATA_PATH, FINAL_DATA_PATH)
#     try:
#         shutil.copy(j, new_path)
#     except FileNotFoundError:
#         os.makedirs(os.path.dirname(new_path), exist_ok=True)
#         shutil.copy(j, new_path)

In [ ]:
import json
import tools.ProcessLv2 as pl2

# for j in json_files:
if True:
    j = '/Users/jinym/Desktop/Desktop_AICenter✨/SFAIcenter/data/FINAL/2C/Lv2/SS0017_low/SS0017.json'
    INPUT_PATH = j
    BACKUP_PATH = INPUT_PATH + ".bak"

    with open(INPUT_PATH, "r", encoding="utf-8") as f:
        origin = json.load(f)

    with open(BACKUP_PATH, "w", encoding="utf-8") as f:
        json.dump(origin, f, ensure_ascii=False, indent=4)

    # id = excel[excel['ISBN'] == int(origin.get('file_id'))].index[0]
    # break
    # id = os.path.splitext(os.path.basename(j))[0]
    # try:
    #     new = {
    #         'file_id': id,
    #         'ISBN': str(excel.loc[id, 'ISBN']),
    #         'title': excel.loc[id, '도서명'],
    #         'cat1_domain': excel.loc[id, '코퍼스 1분류'],
    #         'cat2_sub': excel.loc[id, '코퍼스 2분류'],
    #         'cat3_specific': excel.loc[id, '비고'],
    #         'pub_date': str(excel.loc[id, '출판일'])[:10],
    #         # 'contents': [],
    #         'contents': origin['contents']
    #     }
    # except:
    #     new = origin
    
    new = pl2.fill_chapter(origin)
    # new = pl2.merge_paragraphs(origin)
    # new = pl2.erase_page(origin, 1)
    # new = pl2.format_change(3,origin)
    # new = origin

    # for i in range(len(new['contents'])):
        # contents = new['contents'][i]
        # # c = extract_qna(contents)
        # page_contents = pl2.remove_enter(contents['page_contents'])
        # contents['page_contents'] = page_contents
        # # c = contents

    with open(INPUT_PATH.replace(".json", ".json"), "w", encoding="utf-8") as f:
    # with open(INPUT_PATH.replace(".json", "_new.json"), "w", encoding="utf-8") as f:
        json.dump(new, f, ensure_ascii=False, indent=4)